In [9]:
import torch
import torch.nn as nn

# 定义输入张量，形状为 (batch_size, in_features)
input_tensor = torch.randn(3, 4)  # 3个样本，4个输入特征

# 创建一个线性层
linear_layer = nn.Linear(in_features=4, out_features=2)

# 前向传播
output_tensor = linear_layer(input_tensor)

print("输入张量形状:", input_tensor.shape)
print("权重矩阵形状:", linear_layer.weight.shape)  # 输出权重矩阵的形状
print("偏置项形状:", linear_layer.bias.shape)
print("输出张量形状:", output_tensor.shape)


输入张量形状: torch.Size([3, 4])
权重矩阵形状: torch.Size([2, 4])
偏置项形状: torch.Size([2])
输出张量形状: torch.Size([3, 2])
